In [13]:
import numpy as np
import pandas as pd

from sklearn.naive_bayes import MultinomialNB
from scipy.special import logsumexp

In [3]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer

train = fetch_20newsgroups(subset='train', categories=['alt.atheism', 'talk.religion.misc'])
vectorizer = CountVectorizer(stop_words="english", min_df=5)
vectors = np.asarray(vectorizer.fit_transform(train.data).todense())

In [22]:
class NaiveBayesClassification(object):
    def fit(self, X, y):
        num_class = np.unique(y).shape[0]

        self.prior = np.zeros((num_class))
        for i in range(num_class):
            self.prior[i] = (y == 1).sum() / y.shape[0]
        self.log_prior = np.log(self.prior)

        # Assumming the features are in the representation of bag-of-words
        x_by_c = np.array([X[y == c] for c in range(num_class)]) + 1.0
        sum_words = np.array([arr.sum(0) for arr in x_by_c])
        total_words = sum_words.sum()
        self.log_likelihood = np.log(sum_words / total_words)

    def predict(self, X):
        posterior = np.zeros((X.shape[0], self.prior.shape[0]))
        for i, x in enumerate(X):
            pos = x.astype(bool)
            log_likelihood = self.log_likelihood[:, pos]
            log_likelihood = log_likelihood.sum(1)
            posterior[i] = self.log_prior + log_likelihood
        proba = posterior - logsumexp(posterior, axis=1).reshape(-1, 1)
        return np.exp(proba)

In [23]:
c = NaiveBayesClassification()
c.fit(vectors, train.target)

In [25]:
c.prior, c.log_likelihood

(array([0.43990665, 0.43990665]),
 array([[-8.92654391, -8.8643373 , -8.90808185, ..., -8.93277446,
         -8.91215517, -8.92654391],
        [-9.16376119, -9.13020965, -9.15852557, ..., -9.15331723,
         -9.17166637, -9.14555522]]))

In [26]:
c.predict(vectors)

array([[9.99996866e-01, 3.13446690e-06],
       [9.99999961e-01, 3.85015766e-08],
       [1.00000000e+00, 2.41697085e-11],
       ...,
       [1.00000000e+00, 1.32435020e-18],
       [9.99998718e-01, 1.28170010e-06],
       [1.00000000e+00, 2.27917053e-11]])

In [27]:
bench = MultinomialNB().fit(vectors, train.target)

In [28]:
bench.predict_proba(vectors)

array([[1.00000000e+00, 6.72482499e-30],
       [1.00000000e+00, 3.12984756e-10],
       [1.00000000e+00, 1.10013597e-42],
       ...,
       [1.00000000e+00, 2.06943454e-68],
       [1.00000000e+00, 2.07984869e-21],
       [1.00000000e+00, 3.12470907e-66]])